### Dataset and envs
1. Mount google drive.
2. Declare global variables.
3. Export source code folder.

In [ ]:
GEN_DATA_DIR = '../../vae-dec/data/gene'
LOG_DIR = '../output/log'
MODEL_DIR = '../data/results/model'

# from google.colab import drive
# drive.mount('/content/drive')
# MODEL_DIR = '/content/drive/My Drive/DL/models'
# LOG_DIR = '/content/drive/My Drive/DL/log/vae-dec'
# GEN_DATA_DIR = '/content/drive/My Drive/DL/data/gene/'

#### Import dependencies

In [ ]:
from time import time
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering

# For metagenomics dataset
import itertools as it
import numpy as np
import networkx as nx
# import nxmetis
import copy

from Bio import SeqIO
from Bio.Seq import Seq
import re, os
import gensim
from gensim import corpora

import time

import sys
sys.path.append('../../vae-dec')

In [ ]:
def load_meta_reads(filename, type='fasta'):
    def format_read(read):
        # Return sequence and label
        z = re.split('[|={,]+', read.description)
        return read.seq, z[3]
    try:
        seqs = list(SeqIO.parse(filename, type))
        reads = []
        labels = []

        # Detect for paired-end or single-end reads
        # If the id of two first reads are different (e.g.: .1 and .2), they are paired-end reads
        is_paired_end = False
        if len(seqs) > 2 and seqs[0].id[-1:] != seqs[1].id[-1:]:
            is_paired_end = True

        label_list = dict()
        label_index = 0

        for i in range(0, len(seqs), 2 if is_paired_end else 1):
            read, label = format_read(seqs[i])
            if is_paired_end:
                read2, label2 = format_read(seqs[i + 1])
                read += read2
            reads += [str(read)]

            # Create labels
            if label not in label_list:
                label_list[label] = label_index
                label_index += 1
            labels.append(label_list[label])

        del seqs

        return reads, labels
    except Exception as e:
        print('Error when loading file {} '.format(filename))
        print('Cause: ', e)
        return []

def gen_kmers(klist):
    '''
    Generate list of k-mer words. Given multiple k-mer values.
    Args:
        klist: list of k-mer value
    Return:
        List of k-mer words
    '''
    bases = ['A', 'C', 'G', 'T']
    kmers_list = []
    for k in klist:
        kmers_list += [''.join(p) for p in it.product(bases, repeat=k)]

    # reduce a half of k-mers due to symmetry
    kmers_dict = dict()
    for myk in kmers_list:
        k_reverse_complement=Seq(myk).reverse_complement()
        if not myk in kmers_dict and not str(k_reverse_complement) in kmers_dict:
            kmers_dict[myk]=0

    return list(kmers_dict.keys())

def create_document( reads, klist):
    """
    Create a set of document from reads, consist of all k-mer in each read
    For example:
    k = [3, 4, 5]
    documents =
    [
        'AAA AAT ... AAAT AAAC ... AAAAT AAAAC' - read 1
        'AAA AAT ... AAAT AAAC ... AAAAT AAAAC' - read 2
        ...
        'AAA AAT ... AAAT AAAC ... AAAAT AAAAC' - read n
    ]
    :param reads:
    :param klist: list of int
    :return: list of str
    """
    # create a set of document
    documents = []
    for read in reads:
        k_mers_read = []
        for k in klist:
            k_mers_read += [read[j:j + k] for j in range(0, len(read) - k + 1)]
        documents.append(k_mers_read)

    k_mers_set = [gen_kmers(klist)]
    dictionary = corpora.Dictionary(k_mers_set)
    return dictionary, documents

def save_documents(documents, file_path):
    with open(file_path, 'w') as f:
        for d in documents:
            f.write("%s\n" % d)


def parallel_create_document(reads, klist, n_workers=2 ):
    """
    Create a set of document from reads, consist of all k-mer in each read
    For example:
    k = [3, 4, 5]
    documents =
    [
        'AAA AAT ... AAAT AAAC ... AAAAT AAAAC' - read 1
        'AAA AAT ... AAAT AAAC ... AAAAT AAAAC' - read 2
        ...
        'AAA AAT ... AAAT AAAC ... AAAAT AAAAC' - read n
    ]
    :param reads:
    :param klist: list of int
    :return: list of str
    """

    # create k-mer dictionary
    k_mers_set = [gen_kmers( klist )] #[genkmers(val) for val in klist]
    dictionary = corpora.Dictionary(k_mers_set)

    documents = []
    reads_str_chunk = [list(item) for item in np.array_split(reads, n_workers)]
    chunks = [(reads_str_chunk[i], klist) for i in range(n_workers)]
    pool = Pool(processes=n_workers)

    result = pool.starmap(create_document, chunks)
    for item in result:
        documents += item
    return dictionary, documents

def create_corpus(dictionary: corpora.Dictionary, documents, 
                  is_tfidf=False, 
                  smartirs=None, 
                  is_log_entropy=False, 
                  is_normalize=True):
    corpus = [dictionary.doc2bow(d, allow_update=False) for d in documents]
    if is_tfidf:
        tfidf = TfidfModel(corpus=corpus, smartirs=smartirs)
        corpus = tfidf[corpus]
    elif is_log_entropy:
        log_entropy_model = LogEntropyModel(corpus, normalize=is_normalize)
        corpus = log_entropy_model[corpus]

    return corpus

def compute_kmer_dist(dictionary, corpus, groups, seeds, only_seed=True):
    corpus_m = gensim.matutils.corpus2dense(corpus, len(dictionary.keys())).T
    res = []
    if only_seed:
        for seednodes in seeds:
            tmp = corpus_m[seednodes, :]
            res += [np.mean(tmp, axis=0)]
    else:
        for groupnodes in groups:
            tmp = corpus_m[groupnodes, :]
            res += [np.mean(tmp, axis=0)]
    return np.array(res)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

from utils.utils import *

nmi = normalized_mutual_info_score
ari = adjusted_rand_score


def genome_acc(grps, pred_grps, y_true, n_cluters):
    groups_cluster_lb = assign_cluster_2_reads(grps, pred_grps)
    prec, recall = eval_quality(y_true, groups_cluster_lb, n_clusters=n_cluters)
    f1_score = 2*((prec*recall)/(prec+recall))
    return prec, recall, f1_score
    

def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed

    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

#### Function for creating stable set
1. build_overlap_graph: build a graph of overlapping k-mers.
2. make_stable_set: a greey method for creating groups (connected components) and seeds (independent set).

In [ ]:
def build_overlap_graph(reads, labels, qmer_length, num_shared_reads):
    '''
    Build overlapping graph
    '''
    # Create hash table with q-mers are keys
    print("Building hash table...")
    lmers_dict=dict()
    for idx, r in enumerate(reads):
        for j in range(0,len(r)-qmer_length+1):
            lmer = r[j:j+qmer_length]
            if lmer in lmers_dict:
                lmers_dict[lmer] += [idx]
            else:
                lmers_dict[lmer] = [idx]

    print('Building edges...')
    E=dict()
    for lmer in lmers_dict:
        for e in it.combinations(lmers_dict[lmer],2):
            if e[0]!=e[1]:
                e_curr=(e[0],e[1])
            else:
                continue
            if e_curr in E:
                E[e_curr] += 1 # Number of connected lines between read a and b
            else:
                E[e_curr] = 1
    E_Filtered = {kv[0]: kv[1] for kv in E.items() if kv[1] >= num_shared_reads}
    
    print('Building graph...')
    G = nx.Graph()
    print('Adding nodes...')
    color_map = {0: 'red', 1: 'green', 2: 'blue', 3: 'yellow', 4: 'darkcyan', 5: 'violet'}
    for i in range(0, len(labels)):
        G.add_node(i, label=labels[i], color=color_map[labels[i]])

    print('Adding edges...')
    for kv in E_Filtered.items():
        G.add_edge(kv[0][0], kv[0][1], weight=kv[1])
    print('Graph constructed!')
    
    return G

In [ ]:
def is_seed(neighbors, seed):
    for n_node in neighbors:
        if n_node in seed:
            return False
    return True

def make_stable_set(G: nx.Graph, maximum_seeds):
    GL = []
    SL = []
    temp_G = copy.copy(G)
    traversed_nodes = []
    for i, node in enumerate(temp_G.nodes):
        if node in traversed_nodes:
            continue
        SGi = []
        Gi = []
        SGi.append(node)
        Gi.append(node)
        traversed_nodes.append(node)
        for grp_node in Gi:
            neighbors = list(temp_G.neighbors(grp_node))
            if neighbors:
                add_node = None
                for n_node in neighbors:
                    if n_node not in traversed_nodes:
                        add_node = n_node
                        traversed_nodes.append(n_node)
                        break

                # random_first_neighbor = neighbors[0]
                # # temp_G.remove_node(random_first_neighbor)
                # traversed_nodes.append(random_first_neighbor)
                if add_node:
                    neighbors.remove(add_node)
                    if is_seed(neighbors, SGi):
                        SGi.append(add_node)
                    Gi.append(add_node)

            if (len(SGi) >= maximum_seeds) or (not temp_G.nodes):
                break

        SL.append(SGi)
        GL.append(Gi)

    return GL, SL

#### Define dataset

In [ ]:
import json
from sklearn.preprocessing import OneHotEncoder, normalize, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from dataloader.utils import load_meta_reads, compute_kmer_dist, create_document, create_corpus
class GenomeDataset_v3():
    '''
    Metagenomics dataset for reading simulated data in fasta format (.fna)
    An optimization step based on graph opertation is used to merge reads that
    have overlapping reads into a completed genome.
    '''
    def __init__(self, fna_file, kmers: list, qmers, num_shared_reads, maximum_group_size=5000,
                     maximum_seed_size=200, only_seed=False, is_normalize=True,
                     graph_file=None, is_serialize=False, is_deserialize=False):
        '''
        Args:
            kmers: a list of kmer values. 
            fna_file: path to fna file (fasta format).
            only_seed: only seeds in overlapping graph are used to build features.
            graph_file: calculated groups and seeds (json).
        '''
        # Read fasta dataset
        print('Reading fna file...')
        self.reads, self.labels = load_meta_reads(fna_file, type='fasta')
        # self.reads = self.reads[:5000]
        # self.labels = self.labels[:5000]

        print('Creating document from reads...')
        dictionary, documents = create_document(self.reads, kmers)

        print('Creating corpus...')
        corpus = create_corpus(dictionary, documents)

        self.groups = []
        self.seeds = []

        if is_deserialize:
            print('Deserializing data...')
            self.groups, self.seeds = self.deserialize_data(graph_file, self.reads)
        else:
            # Build overlapping (reads) graph
            print('Building graph from scratch...')
            graph = build_overlap_graph(self.reads, self.labels, qmers, num_shared_reads=num_shared_reads)
            print('Partitioning graph...')
            self.groups, self.seeds = make_stable_set(graph, maximum_seed_size)

        if is_serialize:
            print('Serializing data to...', graph_file)
            self.serialize_data(self.reads, self.groups, self.seeds, graph_file)

        print('Computing features...')
        self.kmer_features = compute_kmer_dist(dictionary, corpus, self.groups, self.seeds, only_seed=only_seed)
        self.groups_kmer_features = compute_kmer_dist(dictionary, corpus, self.groups, self.seeds, only_seed=False)

        if is_normalize:
            print('Normalizing...')
            scaler = StandardScaler()
            self.kmer_features = scaler.fit_transform(self.kmer_features)

        print('Finish.')
    
    def serialize_data(self, reads, groups, seeds, graph_file):
        serialize_dict = {
            'groups': groups,
            'seeds': seeds
        }

        with open(graph_file, 'w') as fg:
            json.dump(serialize_dict, fg)
        
        return graph_file
    
    def deserialize_data(self, graph_file, reads):
        with open(graph_file, 'r') as fg:
            data = json.load(fg)

        groups = data['groups']
        seeds = data['seeds']

        return groups, seeds

#### Model

##### Autoencoder

In [ ]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    x = Input(shape=(dims[0],), name='input')
    h = x

    # internal layers in encoder
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(h)

    # hidden layer
    h = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(h)  # hidden layer, features are extracted from here

    y = h
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        y = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(y)

    # output
    y = Dense(dims[0], kernel_initializer=init, name='decoder_0')(y)

    return Model(inputs=x, outputs=y, name='AE'), Model(inputs=x, outputs=h, name='encoder')

##### Clustering layer

In [ ]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

##### Main model

In [ ]:
class DEC(object):
    def __init__(self,
                 dims,
                 n_clusters=10,
                 alpha=1.0,
                 init='glorot_uniform'):

        super(DEC, self).__init__()

        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.autoencoder, self.encoder = autoencoder(self.dims, init=init)

        # prepare DEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(self.encoder.output)
        self.model = Model(inputs=self.encoder.input, outputs=clustering_layer)

    def pretrain(self, x, y=None, grps=None, n_clusters=2, optimizer='adam', epochs=200, batch_size=256, save_dir='results/temp'):
        print('...Pretraining...')
        self.autoencoder.compile(optimizer=optimizer, loss='mse')

        csv_logger = callbacks.CSVLogger(save_dir + '/pretrain_log.csv')
        cb = [csv_logger]
        if y is not None:
            class PrintACC(callbacks.Callback):
                def __init__(self, x, y):
                    self.x = x
                    self.y = y
                    super(PrintACC, self).__init__()

                def on_epoch_end(self, epoch, logs=None):
                    if int(epochs/10) != 0 and epoch % int(epochs/10) != 0:
                        return
                    feature_model = Model(self.model.input,
                                          self.model.get_layer(
                                              'encoder_%d' % (int(len(self.model.layers) / 2) - 1)).output)
                    features = feature_model.predict(self.x)
                    km = KMeans(n_clusters=len(np.unique(self.y)), n_init=20, n_jobs=4)
                    y_pred = km.fit_predict(features)
                    # print()
                    # print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                    #       % (metrics.acc(self.y, y_pred), metrics.nmi(self.y, y_pred)))

                    print(' '*8 + '|==>  acc: %.4f <==|' % (genome_acc(grps, y_pred, self.y, n_clusters)[1]))
            cb.append(PrintACC(x, y))

        # begin pretraining
        t0 = time.time()
        self.autoencoder.fit(x, x, batch_size=batch_size, epochs=epochs, callbacks=cb)
        print('Pretraining time: %ds' % round(time.time() - t0))
        self.autoencoder.save_weights(save_dir + '/ae_weights.h5')
        print('Pretrained weights are saved to %s/ae_weights.h5' % save_dir)
        self.pretrained = True

    def load_weights(self, weights):  # load weights of DEC model
        self.model.load_weights(weights)

    def extract_features(self, x):
        return self.encoder.predict(x)

    def predict(self, x):  # predict cluster labels using the output of clustering layer
        q = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def compile(self, optimizer='sgd', loss='kld'):
        self.model.compile(optimizer=optimizer, loss=loss)

    def fit(self, x, y=None, grps=None, n_clusters=2, maxiter=2e4, batch_size=256, tol=1e-3,
            update_interval=140, save_dir='./results/temp'):

        best_model = self.model
        best_acc = 0.0
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        print('Update interval', update_interval)
        save_interval = int(x.shape[0] / batch_size) * 5  # 5 epochs
        print('Save interval', save_interval)

        # Step 1: initialize cluster centers using k-means
        t1 = time.time()
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # Step 2: deep clustering
        # logging file
        import csv
        logfile = open(save_dir + '/dec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'precision', 'recall', 'f1_score', 'nmi', 'ari', 'loss'])
        logwriter.writeheader()

        loss = 0
        index = 0
        index_array = np.arange(x.shape[0])
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                if y is not None:
                    # acc = np.round(metrics.acc(y, y_pred), 5)
                    # nmi = np.round(metrics.nmi(y, y_pred), 5)
                    # ari = np.round(metrics.ari(y, y_pred), 5)
                    prec, recall, f1_score = genome_acc(grps, y_pred, y, n_clusters)
                    if f1_score > best_acc:
                      best_acc = f1_score
                      best_model = self.model
                    loss = np.round(loss, 5)
                    # logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, loss=loss)
                    logdict = dict(iter=ite, precision=prec, recall=recall, f1_score=f1_score, loss=loss)
                    logwriter.writerow(logdict)
                    # print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)
                    print('Iter %d: precision = %.5f, recall = %.5f, f1_score = %.5f,\
                            nmi = --, ari = --' % (ite, prec, recall, f1_score), ' ; loss=', loss)

                # check stop criterion
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            # if index == 0:
            #     np.random.shuffle(index_array)
            idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
            loss = self.model.train_on_batch(x=x[idx], y=p[idx])
            index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

            # save intermediate model
            if ite % save_interval == 0:
                print('saving model to:', save_dir + '/DEC_model_' + str(ite) + '.h5')
                self.model.save_weights(save_dir + '/DEC_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        logfile.close()
        print('saving model to:', save_dir + '/DEC_model_final.h5')
        best_model.save_weights(save_dir + '/DEC_model_final.h5')

        return y_pred

#### Dataset loader

In [ ]:
def load_genomics(dataset_name,
                    kmers, 
                    lmer,
                    maximum_seed_size,
                    num_shared_reads,
                    graph_file=None,
                    is_serialize=False,
                    is_deserialize=False,
                    is_normalize=False,
                    only_seed=False):
    '''
    Loads fna file.
    Args:
        dataset_name: name of dataset (e.g. S1.fna, L1.fna,...)
        kmers: list of kmers.
        lmer: lmer.
        maximum_seed_size.
        num_shared_reads.
        graph_file: computed groups/seeds json file.
        is_serialize: True to serialize computed groups/seeds to json file.
        is_deserialize: True to load computed groups/seeds in json file.
        is_normalize: whether to normalize kmer-features.
        only_seed: True to compute kmer features using seeds only.
    '''
    genomics_dataset = GenomeDataset_v3(
        dataset_name, kmers, lmer,
        graph_file=graph_file,
        only_seed=only_seed,
        maximum_seed_size=maximum_seed_size,
        num_shared_reads=num_shared_reads,
        is_serialize=is_serialize,
        is_deserialize=is_deserialize,
        is_normalize=is_normalize)

    return genomics_dataset.kmer_features,\
        genomics_dataset.groups_kmer_features,\
        genomics_dataset.labels,\
        genomics_dataset.groups,\
        genomics_dataset.seeds

#### Inspects stable set

##### TSNE visualizing

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import time, os

In [ ]:
def visualize(x, labels, n_clusters, range_lim=(-20, 20), perplexity=40):
    tsne = TSNE(n_components=n_clusters, verbose=1, perplexity=perplexity, n_iter=400, init='pca')
    tsne_results = tsne.fit_transform(x)
    df_subset = pd.DataFrame()
    df_subset['tsne-2d-one'] = tsne_results[:,0]
    df_subset['tsne-2d-two'] = tsne_results[:,1]
    df_subset['Y'] = labels
    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x='tsne-2d-one', y='tsne-2d-two',
        hue='Y',
        palette=sns.color_palette(n_colors=n_clusters),
        data=df_subset,
        legend="full",
        alpha=0.3
    ).set(xlim=range_lim,ylim=range_lim)

#### Training parts

In [ ]:
BATCH_SIZE = 256
MAX_ITERS = 1000
PRETRAIN_EPOCHS = 20
UPDATE_INTERVAL = 30
TOL = 0.0001

AE_WEIGHTS = None
AE_WEIGHTS = '../data/results/model\dec-keras\gen\S1/ae_weights.h5'
DATASET_DIR = GEN_DATA_DIR
DATASET_NAME = 'S1'
SAVE_DIR = os.path.join(MODEL_DIR, 'dec-keras', 'gen', DATASET_NAME)

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Hyperparameters
KMERS = [4]
N_CLUSTERS = 2
LMER = 20
NUM_SHARED_READS = 5
ONLY_SEED = True
MAXIMUM_SEED_SIZE = 5000

In [ ]:
seed_kmer_features, group_kmer_features, labels, groups, seeds = load_genomics(
        os.path.join(DATASET_DIR, DATASET_NAME + '.fna'),
        kmers=KMERS,
        lmer=LMER,
        maximum_seed_size=MAXIMUM_SEED_SIZE,
        num_shared_reads=NUM_SHARED_READS,
        is_deserialize=True,
        is_serialize=True,
        is_normalize=True,
        only_seed=ONLY_SEED,
        graph_file=os.path.join(DATASET_DIR, 'processed', DATASET_NAME + '.json')
    )

In [ ]:
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                               distribution='uniform')  # [-limit, limit], limit=sqrt(1./fan_in)
pretrain_optimizer = SGD(lr=1.0, momentum=0.9)
dec = DEC(dims=[seed_kmer_features.shape[-1], 500, 500, 2000, 10], n_clusters=N_CLUSTERS, init=init)

In [ ]:
if AE_WEIGHTS is None:
    dec.pretrain(x=seed_kmer_features, y=labels, grps=groups, optimizer=pretrain_optimizer,
                epochs=PRETRAIN_EPOCHS, batch_size=BATCH_SIZE,
                save_dir=SAVE_DIR)
else:
    dec.autoencoder.load_weights(AE_WEIGHTS)

dec.model.summary()
t0 = time.time()
dec.compile(optimizer=SGD(0.1, 0.9), loss='kld')
y_pred = dec.fit(x=seed_kmer_features, y=labels, grps=groups, n_clusters=N_CLUSTERS, tol=TOL, maxiter=MAX_ITERS, batch_size=BATCH_SIZE,
                  update_interval=UPDATE_INTERVAL, save_dir=SAVE_DIR)
print('acc:', genome_acc(groups, y_pred, labels, N_CLUSTERS)[1])
print('clustering time: ', (time.time() - t0))

#### Visualize

In [ ]:
# Assign groups labels from reads labels
# Dominant number of each label is the label of group
# [0 0 0 0 1 1 0] -> group has label 0 (5 > 2)
grps_label = []
for group in groups:
    lb_type = {}
    for node in group:
        if labels[node] not in list(lb_type.keys()):
            lb_type[labels[node]] = 1
        else:
            lb_type[labels[node]] += 1
    max_val = 0
    key = -1
    for kv in lb_type.items():
        if kv[1] > max_val:
            max_val = kv[1]
            key = kv[0]
    if key != -1:
        grps_label.append(key)

##### Seeds

In [ ]:
visualize(seed_kmer_features, grps_label, N_CLUSTERS, (-20, 20), 40)

##### Groups

In [ ]:
visualize(group_kmer_features, grps_label, N_CLUSTERS, (-20, 20), 40)

##### Gtruth of latent space

In [ ]:
latent = dec.encoder.predict(seed_kmer_features)
visualize(latent, grps_label, N_CLUSTERS, (-50, 50), 20)

##### Prediction

In [ ]:
latent = dec.encoder.predict(seed_kmer_features)
y_pred = dec.predict(seed_kmer_features)
visualize(latent, y_pred, N_CLUSTERS, (-50, 50), 20)